# Preprocessing

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
# Extracting the data 

from unstructured.partition.pdf import partition_pdf

output_path = ''
file_path = '/Users/felipesilverio/Documents/GitHub/LangChainTest/test2.pdf'

chunks = partition_pdf(
    filename= file_path,
    infer_table_structure=True, #extracting table
    strategy = 'hi_res', #mandatory to infer table

    extract_image_block_types=['Image'], #add 'Table' to list to extract image of tables
    # image_output_dir_path = output_path, #if None, images and tables will be saved as base64

    extract_image_block_to_payload=True, #if true, extract base64 for API usage

    chunking_strategy='by_title', #or basic
    max_characters=10000, #default is 500
    combine_text_under_n_chars=2000, #default is 0
    new_after_n_chars=6000, #default is 0
)

chunks

/Users/felipesilverio/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The `max_size` parameter is deprecated and will be removed in v4.26. Please specify in `size['longest_edge'] instead`.


In [31]:
tables, texts, images, grouped_texts = [], [], [], []

for chunk in chunks:
    # Work on the inner elements list (or the chunk itself if not composite)
    element_list = (
        chunk.metadata.orig_elements
        if "CompositeElement" in str(type(chunk))
        else [chunk]
    )

    idx = 0
    while idx < len(element_list):
        el = element_list[idx]

        if "Table" in str(type(el)):
            tables.append(el)

        elif "Image" in str(type(el)):
            images.append(el)

        # ── Title + next-element grouping ───────────────────────────────────
        elif "Title" in str(type(el)) and idx + 1 < len(element_list):
            nxt = element_list[idx + 1]
            if nxt.category not in ("Title", "Table", "Image"):
                grouped_texts.append(f"{el.text}\n{nxt.text}")
                idx += 2          # skip the element we just merged
                continue
            else:
                texts.append(el)  # title without suitable follower
        # ────────────────────────────────────────────────────────────────────
        else:
            texts.append(el)

        idx += 1


In [66]:
tables

In [ ]:
tables, texts, images= [], [], []

for chunk in chunks:
    if "CompositeElement" in str(type(chunk)):
        chunk_els = chunk.metadata.orig_elements
        for el in chunk_els:
            if "Table" in str(type(el)):
                tables.append(el)
            elif "Image" in str(type(el)):
                images.append(el)
            elif "Title" in str(type(el)) and i + 1 < len(chunks):
                nxt = chunks[i + 1]
                if nxt.category not in ("Title", "Table", "Image"):
                    combined = f"{el.text}\n{nxt.text}"
                    grouped_texts.append(combined)
                    i += 2
                    continue  # skip to element after next
            else:
                texts.append(el)

In [35]:
import uuid
from langchain.vectorstores import Chroma
from langchain.storage import InMemoryStore
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings
from langchain.retrievers.multi_vector import MultiVectorRetriever

vectorstore = Chroma(collection_name='multi_modal_rag', embedding_function=OpenAIEmbeddings())

#Storage layor
store = InMemoryStore()
id_key = 'doc_id'

#retriever

retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    docstore=store,
    id_key=id_key,
)

In [67]:
doc_ids = [str(uuid.uuid4()) for _ in grouped_texts]

text_vectors = [
    Document(page_content=txt, metadata={id_key: uid, "type": "text"})
    for txt, uid in zip(grouped_texts, doc_ids)
]
retriever.vectorstore.add_documents(text_vectors)

# store full Documents (not plain strings!)
retriever.docstore.mset(
    (uid, Document(page_content=txt, metadata={"type": "text"}))
    for uid, txt in zip(doc_ids, grouped_texts)
)

In [68]:
table_ids = [str(uuid.uuid4()) for _ in tables]
table_strings = [tbl.text for tbl in tables]                  # convert to str once

table_docs = [
    Document(page_content=tbl_txt, metadata={id_key: uid, "type": "table"})
    for tbl_txt, uid in zip(table_strings, table_ids)
]

retriever.vectorstore.add_documents(table_docs)
retriever.docstore.mset(list(zip(table_ids, table_strings))) 

In [59]:
retriever.docstore.mset(
    (uid, Document(page_content=txt, metadata={"type": "text"}))
    for uid, txt in zip(doc_ids, grouped_texts)
)

# --- store tables -----------------------------------------------------------
retriever.docstore.mset(
    (uid, Document(page_content=tbl_txt, metadata={"type": "table"}))
    for uid, tbl_txt in zip(table_ids, table_strings)
)

In [ ]:
# #Loading values

# doc_ids = [str(uuid.uuid4()) for _ in grouped_texts]
# original_text_docs = [
#     Document(page_content=text, metadata={id_key: doc_ids[i]}) for i, text in enumerate(grouped_texts)
# ]
# retriever.vectorstore.add_documents(original_text_docs)
# retriever.docstore.mset(list(zip(doc_ids, grouped_texts)))

# # Add tables
# tables_ids = [str(uuid.uuid4()) for _ in tables]
# original_table_docs = [
#     Document(page_content=table, metadata={id_key: tables_ids[i]}) for i, table in enumerate(tables)
# ]
# retriever.vectorstore.add_documents(original_table_docs)
# retriever.docstore.mset(list(zip(tables_ids, tables)))



# RAG

In [69]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_openai import ChatOpenAI
from base64 import b64decode
from prompts import system_finance_prompt
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# def parse_docs(docs):
#     # Split base64 images and texts
#     b64 = []
#     text = []
#     for doc in docs:
#         try:
#             b64decode(doc)
#             b64.append(doc)
#         except Exception as e:
#             text.append(doc)
#     return{'images':b64, 'texts':text}

def parse_docs(docs):
    images, texts = [], []
    for doc in docs:                       # doc: langchain_core.documents.Document
        payload = doc.page_content         # <-- the actual string you stored
        try:
            b64decode(payload, validate=True)
            images.append(payload)         # looks like a base-64 image
        except Exception:
            texts.append(payload)          # plain text / table string
    return {"images": images, "texts": texts}


def build_prompt_two(kwargs) -> ChatPromptTemplate:
    """
    Construct a ChatPromptTemplate that always begins with the system prompt,
    then includes context (text + images) and the user question.
    """

    context = kwargs['context']
    question = kwargs['question']

    # Concatenate all text fragments
    # context_text = "".join([t.text for t in context.get('texts', [])])
    context_text = "\n".join(context.get("texts", []))
    # Build the messages list: SystemMessage -> HumanMessage
    messages = [
        SystemMessage(content=system_finance_prompt),
        HumanMessage(content=f"Context: {context_text}\nQuestion: {question}")
    ]

    # Include images if present
    for b64 in context.get('images', []):
        messages.append(
            HumanMessage(content={
                'type': 'image_url',
                'image_url': {'url': f'data:image/jpeg;base64,{b64}'},
            })
        )

    # Create and return a prompt template from these messages
    return ChatPromptTemplate.from_messages(messages)


chain = (
    {
        'context': retriever | RunnableLambda(parse_docs),
        'question': RunnablePassthrough(),
    }
    | RunnableLambda(build_prompt_two)
    | ChatOpenAI(model='o3')
    | StrOutputParser()
)

chain_with_sources = {
    'context': retriever | RunnableLambda(parse_docs),
    'question': RunnablePassthrough(),
} | RunnablePassthrough().assign(
    response=(
        RunnableLambda(build_prompt_two)
        | ChatOpenAI(model='o3')
        | StrOutputParser()
    )
)

In [70]:
print("Total embeddings in collection:", len(vectorstore._collection.get()["ids"]))


Total embeddings in collection: 999


In [71]:
vectorstore._collection.get(include=["embeddings"])["embeddings"][0][:5]

array([-0.01645603, -0.0205357 ,  0.01218406, -0.01418955, -0.02765108])

In [73]:
docs = retriever.get_relevant_documents("Tour Partner Group Limited")
print(len(docs), "docs retrieved")
for d in docs[:3]:
    print(d.page_content[:120], "…")

0 docs retrieved


In [74]:
# my_prompt = 'Make the company profile of Tour Partner Group Limited. Even if it is not available: revenue split by geography or segment, full cash-flow statement or EBITDA reconciliation, or other problems.'
my_prompt = """ Make the company profile of Tour Partner Group Limited. """
profile = chain_with_sources.invoke(my_prompt)
print(profile)

{'context': {'images': [], 'texts': []}, 'question': ' Make the company profile of Tour Partner Group Limited. ', 'response': 'I’m very happy to put together the full restructuring-style one-pager for Tour Partner Group Ltd., however the profile needs the exact figures and wording that appear in the company’s most recent statutory Annual Report (e.g. FY22 or FY23) for every table and most commentary sections.  \n\nThat U.K. filing is normally available on Companies House as a PDF, but I don’t have direct access to that document within this chat.  Could you please upload or copy-paste the relevant pages (income statement, cash-flow statement, balance sheet and segmental revenue note), or provide the full FY22 / FY23 Annual Report?  \n\nOnce I have those source pages, I can produce the fully-sourced profile with all the required accuracy.'}


In [52]:
print(profile['response'])

I’m very happy to prepare the full restructuring one-pager, but I need a copy (PDF or image) of Tour Partner Group Limited’s most recent annual report (or at least its primary financial statements and notes covering segment/geographic revenue, debt facilities, cash-flow statement and equity information).  

Without that source document I cannot meet the required accuracy standard for:  
• exact employee count and operational KPIs  
• revenue breakdown that exactly ties to reported totals  
• three-year financial highlights table (revenue, margins, cash-flow items, leverage etc.)  
• detail on debt facilities, maturity schedule, covenants and liquidity  

If you can upload or link the FY23 (or FY22/FY24) statutory accounts filed at Companies House—or any management presentation or interim report that contains the same data—I can immediately build the complete profile with full source citations.


# PDF and PPT

In [34]:
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.pagesizes import letter
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from pptx import Presentation
from pptx.util import Inches
from PyPDF2 import PdfReader


In [46]:
def text_to_pdf_unicode(text: str, output_path: str):
    """
    Creates a PDF file from plain text with full Unicode support
    using ReportLab's Platypus framework.
    
    Parameters:
    - text: the string content to write into the PDF.
    - output_path: full file path where the PDF will be saved.
    """
    # 1. Prepare the document
    doc = SimpleDocTemplate(output_path, pagesize=letter)
    styles = getSampleStyleSheet()
    body_style = styles['BodyText']
    
    # 2. Build a "story" of flowable objects
    story = []
    for line in text.split('\n'):
        # Paragraph handles Unicode (e.g. “–”, “é”, emojis, etc.) natively
        story.append(Paragraph(line or ' ', body_style))
        # Small spacer between lines
        story.append(Spacer(1, 4))
    
    # 3. Generate the PDF
    doc.build(story)

profile_text = profile["response"]  # fetched from your chain output
text_to_pdf_unicode(profile_text, "/Users/felipesilverio/Documents/GitHub/LangChainTest/output/company_profile_unicode2.pdf")


In [36]:
from pptx.util import Pt
from pptx import Presentation
from pptx.enum.shapes import MSO_SHAPE_TYPE
from typing import Any, Dict, List, Optional

def replace_placeholder_text(file_path: str,placeholder: str,replacement: str,output_path: Optional[str] = None) -> None:
    """
    Replace occurrences of `placeholder` in text elements with `replacement`.

    Modifies the presentation and saves to `output_path` or overwrites original.
    """
    try:
        prs = Presentation(file_path)
    except Exception as e:
        raise IOError(f"Unable to open file {file_path}: {e}")

    for slide in prs.slides:
        for shape in slide.shapes:
            if not shape.has_text_frame:
                continue
            for paragraph in shape.text_frame.paragraphs:
                for run in paragraph.runs:
                    if placeholder in run.text:
                        run.text = run.text.replace(placeholder, replacement)
                        run.font.size = Pt(5)

    save_path = output_path or file_path
    try:
        prs.save(save_path)
    except Exception as e:
        raise IOError(f"Unable to save updated file to {save_path}: {e}")


In [39]:
from PyPDF2 import PdfReader

reader = PdfReader('/Users/felipesilverio/Documents/GitHub/LangChainTest/output/company_profile_unicode.pdf')
reader

all_text = []
for page_num, page in enumerate(reader.pages, start=1):
    try:
        text = page.extract_text() or ''
    except Exception:
        text = ''
    all_text.append(text)

pdf_retrieved = "\n".join(all_text)
pdf_retrieved

'Company Profile – Tour Partner Group Limited\nSources utilised:\n(1) Companies House – “Tour Partner Group Limited, Annual Report & Financial Statements for the\nyear-ended 31-Dec-22”, filed 29-Sep-23 (pdf)\n(2) Tour Partner Group website – “About Us” page, accessed 07-Jun-24\n--------------------------------------------------\n1. Company Snapshot\nPrimary Industry: Travel & Tourism\nIncorporation Year: 2016\nHeadquarters: London, United Kingdom\nEmployees: 230 (FY22, Companies House filing, note 6)\nOperational KPIs (FY22)\n\x7f Passengers handled: 350,000\n\x7f Bed-nights booked: 710,000\n\x7f Source markets served: 70+\n(Sources: (1) Directors’ report & strategic review; (2) corporate website)\n--------------------------------------------------\n2. Business Overview (bullets only)\n\x7f Tour Partner Group Limited is an intermediate holding company that consolidates a portfolio of B2B\ndestination management and group travel brands serving the UK & Ireland, the Nordics and\nContinen

In [47]:
ppt_path = '/Users/felipesilverio/Documents/GitHub/LangChainTest/try2.pptx'
my_prompt = f"""
The following text contains a series of text blocks that are separated by multiples - like:
--------------------------------------------------------------------------------------------------------------------

Bring me back only the code block relative to 1. Company Snapshot, and give me all the information in it in a single line but separated by |

The text is:

{pdf_retrieved}
"""
response = chain_with_sources.invoke(my_prompt)
replace_placeholder_text('/Users/felipesilverio/Documents/GitHub/LangChainTest/backupppt.pptx', 'Company Snapshot', response['response'], ppt_path)

my_prompt = f"""
The following text contains a series of text blocks that are separated by multiples - like:
--------------------------------------------------------------------------------------------------------------------

Bring me back only the code block relative to Business Overview, and give me back exactly as is in the text.

The text is:

{pdf_retrieved}
"""
response = chain_with_sources.invoke(my_prompt)
replace_placeholder_text('/Users/felipesilverio/Documents/GitHub/LangChainTest/backupppt.pptx', 'Business Overview Text', response['response'], ppt_path)
